In [ ]:
# Data Extraction
import requests, pandas as pd

RAW_URL = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"

resp = requests.get(RAW_URL, timeout=30)
resp.raise_for_status()

data = resp.json()
df = pd.json_normalize(data)
print(df.head(), df.shape)

In [ ]:
# Knowinwg Data Sets
print(df.shape)
print(df.dtypes)
print(df.head())
df.info()

In [ ]:
unique_values_dict = {}
for column in df.columns:
  unique_values_dict[column] = pd.unique(df[column])

for column, values in unique_values_dict.items():
    print(f"Unique values in '{column}': {values}")

In [ ]:
print(df.isna().sum())
(df.isna().mean() * 100).round(2)

In [ ]:
df["account.Charges.Total"] = pd.to_numeric(df["account.Charges.Total"], errors="coerce")
print(df["account.Charges.Total"].isna().sum())
print(df["account.Charges.Total"].describe())

In [ ]:
missing_rows = df[df["account.Charges.Total"].isna()]
print(missing_rows)

In [ ]:
df.loc[df["customer.tenure"] == 0, "account.Charges.Total"] = 0

In [18]:
df.duplicated().sum()
df["customer.gender"].value_counts()
df = df[df["Churn"].str.strip() != ""]


In [ ]:
df["account.Daily.Cuentas_Diarias"] = df["account.Charges.Monthly"] / 30
print(df[["account.Charges.Monthly", "account.Daily.Cuentas_Diarias"]])

In [ ]:
df.describe(include="all")

In [ ]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
print(df[num_cols].describe().T)

In [ ]:
cat_cols = df.select_dtypes(include=["object"]).columns
for col in cat_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts(normalize=True).round(2) * 100)

In [ ]:
for col in ["customer.gender", "customer.Partner", "customer.Dependents",
            "phone.PhoneService", "internet.InternetService", "account.Contract"]:
    print(f"\nChurn rate by {col}:")
    print(df.groupby(col)["Churn"].value_counts(normalize=True).unstack().round(2)*100)


In [ ]:
# Distribucion de Evasion
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.countplot(x="Churn", data=df, palette="pastel")
plt.title("Customer Churn Distribution (Count)")
plt.xlabel("Churn")
plt.ylabel("Number of Customers")
plt.show()

churn_counts = df["Churn"].value_counts()
labels = churn_counts.index
sizes = churn_counts.values
colors = ["#66b3ff","#ff9999"]

plt.figure(figsize=(5,5))
plt.pie(sizes, labels=labels, autopct="%.1f%%", colors=colors, startangle=90)
plt.title("Customer Churn Distribution (%)")
plt.show()

churn_rate = df["Churn"].value_counts(normalize=True) * 100

plt.figure(figsize=(6,4))
sns.barplot(x=churn_rate.index, y=churn_rate.values, palette="pastel")
plt.title("Customer Churn Distribution (%)")
plt.ylabel("Percentage of Customers")
plt.xlabel("Churn")
for i, val in enumerate(churn_rate.values):
    plt.text(i, val+1, f"{val:.1f}%", ha="center", fontsize=12)
plt.show()



In [ ]:
# Recuento de evasión por variables categóricas
## Churn Por Genero
plt.figure(figsize=(6,4))
sns.countplot(x="customer.gender", hue="Churn", data=df, palette="pastel")
plt.title("Churn Distribution by Gender")
plt.show()

## Churn por tipo de contrato
plt.figure(figsize=(7,4))
sns.countplot(x="account.Contract", hue="Churn", data=df, palette="pastel")
plt.title("Churn Distribution by Contract Type")
plt.show()

## Churn por metodo de pago
plt.figure(figsize=(8,4))
sns.countplot(x="account.PaymentMethod", hue="Churn", data=df, palette="pastel")
plt.xticks(rotation=30)
plt.title("Churn Distribution by Payment Method")
plt.show()

## Churn por servicio de internet
plt.figure(figsize=(7,4))
sns.countplot(x="internet.InternetService", hue="Churn", data=df, palette="pastel")
plt.title("Churn Distribution by Internet Service")
plt.show()




In [ ]:
# Conteo de evasión por variables numéricas

## Tenure vs Churn
plt.figure(figsize=(7,4))
sns.histplot(data=df, x="customer.tenure", hue="Churn", bins=30, multiple="stack", palette="pastel")
plt.title("Tenure Distribution by Churn")
plt.xlabel("Months with Company")
plt.show()

## Monthly Charges vs Churn
plt.figure(figsize=(7,4))
sns.boxplot(x="Churn", y="account.Charges.Monthly", data=df, palette="pastel")
plt.title("Monthly Charges Distribution by Churn")
plt.show()

## Total Charges vs Churn
plt.figure(figsize=(7,4))
sns.boxplot(x="Churn", y="account.Charges.Total", data=df, palette="pastel")
plt.title("Total Charges Distribution by Churn")
plt.show()



### Informe de Análisis de Churn

#### Introducción

Este informe analiza la rotación de clientes (churn), que es la proporción de clientes que cancelan su servicio, y los factores asociados a ella. Los objetivos son:

- Limpiar y preparar el conjunto de datos para un análisis fiable.
    
- Explorar la distribución de la rotación en general y a través de características categóricas clave (género, contrato, método de pago, servicios).
    
- Comparar las características numéricas (permanencia, cargos mensuales, cargos totales) entre clientes que rotaron y los que no.
    
- Resumir los hallazgos y proponer recomendaciones viables para reducir la rotación.
    

#### Limpieza y Procesamiento de Datos

- Se eliminaron los espacios en blanco de las columnas de texto.
    
- Se convirtió `account.Charges.Total` a tipo numérico (los valores no válidos se convirtieron a `NaN`).
    
- Se estandarizaron las etiquetas de rotación (`Sí`/`No`); se eliminaron las filas con valores de rotación en blanco o faltantes.
    
- Se estableció `account.Charges.Total = 0` para los clientes con una permanencia de 0 (clientes nuevos).
    
- Se eliminaron las filas con valores faltantes restantes en `account.Charges.Total`.
    
- Se creó `account.Daily_Accounts` = `account.Charges.Monthly` / 30.
    
- Se eliminaron las filas duplicadas.
    
- **Resultado:** El conjunto de datos está limpio y consistente, con la información de rotación estandarizada.
    

#### Análisis Exploratorio de Datos

**Distribución General de la Rotación**

- Tasa de rotación general: ~26% (clientes que rotaron) vs ~74% (clientes que no rotaron).
    
- El conjunto de datos está desequilibrado, con más clientes que se quedan que los que se van.
    

**Rotación por Características Categóricas**

- **Tipo de Contrato:**
    
    - Los clientes con contrato mensual rotan más (~40% de rotación).
        
    - Los contratos de 1 y 2 años tienen una rotación mucho menor (~10%).
        
- **Método de Pago:**
    
    - Los clientes que pagan con cheque electrónico tienen la rotación más alta.
        
    - Los métodos de pago automático (tarjeta de crédito, transferencia bancaria) muestran una rotación mucho menor.
        
- **Servicio de Internet:**
    
    - Los usuarios de fibra óptica muestran una rotación más alta que los clientes de DSL o los que no tienen servicio de internet.
        
- **Género, Pareja, Dependientes:**
    
    - Tienen un efecto insignificante o nulo en la rotación.
        

**Características Numéricas vs. Rotación**

- **Permanencia (`Tenure`):**
    
    - Los clientes que rotaron suelen tener una permanencia baja (mediana de ~1 año).
        
    - Los clientes que no rotaron tienen una permanencia más larga (mediana de ~3-4 años).
        
- **Cargos Mensuales (`Monthly Charges`):**
    
    - Los clientes que rotaron pagan cargos mensuales más altos (mediana de ~$80).
        
    - Los clientes que no rotaron pagan cargos mensuales más bajos (mediana de ~$65).
        
- **Cargos Totales (`Total Charges`):**
    
    - Los clientes que rotaron acumulan un gasto total mucho menor, lo que es consistente con una permanencia más corta.
        

#### Conclusiones e Ideas Clave

- **Tasa de rotación general:** Aproximadamente 1 de cada 4 clientes cancela el servicio.
    
- El **tipo de contrato** es el factor más influyente en la rotación: los contratos a corto plazo tienen un alto riesgo.
    
- Los **métodos de pago** son importantes: los usuarios de cheque electrónico son significativamente más propensos a irse.
    
- El **nivel de servicio** influye en la rotación: los clientes de fibra óptica rotan más (posiblemente relacionado con el precio o la calidad).
    
- La **permanencia** es fundamental: los clientes nuevos rotan mucho más; los clientes con una larga permanencia son leales.
    
- La **presión de los precios**: las facturas mensuales más altas están correlacionadas con una mayor rotación.
    

#### Recomendaciones

- **Promover contratos más largos:** Ofrecer descuentos o beneficios a los clientes que pasen de planes mensuales a anuales.
    
- **Fomentar métodos de pago estables:** Incentivar la migración de cheques electrónicos a métodos de pago automático.
    
- **Programas de retención temprana:** Centrarse en los primeros 90 días con una buena incorporación, soporte proactivo y ofertas de fidelización.
    
- **Dirigirse a clientes con cargos altos:** Ofrecer la optimización del plan, paquetes o descuentos de fidelización para reducir la rotación impulsada por el precio.
    
- **Centrarse en la calidad del servicio:** Prestar atención a los clientes de fibra óptica (mayor rotación) con monitoreo de calidad, comunicación proactiva y soporte.